In [5]:
import sys
!conda install --yes --prefix {sys.prefix} beautifulsoup4

Solving environment: done

# All requested packages already installed.



In [7]:
import beautifulsoup4

ModuleNotFoundError: No module named 'beautifulsoup4'

## Meeting Plan for Nicolas Ouellet and Benedicte Knudson:
We colaborated over Zoom and in person **blank** times and used a shared Github Repository that can be found [here](https://github.com/NickOuellet/Fly-Me-To-The-Moon)